# Hapmap Generator
## By Samuel Horovatin, s.horovatin@usask.ca

A simplistic hapmap generator. Follows the format outlined here: http://augustogarcia.me/statgen-esalq/Hapmap-and-VCF-formats-and-its-integration-with-onemap/

In [1]:
import os, sys
import pandas as pd
import numpy as np

# Change to path of unprocessed hapmap. Format should be: Index, Name, Traits.....
RAWHAP = "./hapmaps/wheat_hapmap_new.txt"

# Change to path of 90K summary. Found mine at: :https://urgi.versailles.inra.fr/download/iwgsc/IWGSC_RefSeq_Annotations/v1.0/, zip file: iwgsc_refseqv1.0_Marker_mapping_summary_2017Mar13/infinium90K.summary.gff)
# Made a slight edit to the raw summary to add headers (chrom	1	2	pos1	pos2	3	strand	4	other)
SUMMARY90K = "./hapmaps/infinium90K.summary.gff"

# Change to output file location/name
OUTPUT = "./hapmaps/wheat_hapmap_gen.txt"

# Column headers used within the fromated hapmap
COLHEADERS = ['rs#','alleles','chrom','pos','strand','assembly#','center', 'protLSID', 'assayLSID', 'panelLSID', 'QCcode']

# Column headers for none SNP columns used in the RAWHAP
SUMMARYHEADERS = ['Index', 'Name']

In [2]:
# Load in the relevant data
raw_hap_df = pd.read_csv(RAWHAP, sep='\t')
summary_90k_df = pd.read_csv(SUMMARY90K, sep='\t')

In [3]:
# Splits other column into distinct columns and does a touch of trimming
summary_90k_df[['ID', 'Name', 'coverage', 'identity']] = summary_90k_df['other'].str.split(';',expand=True) 
summary_90k_df['chrom'] = summary_90k_df['chrom'].map(lambda x: "".join(filter(str.isdigit, x)))
summary_90k_df['ID'] = summary_90k_df['ID'].map(lambda x: x.replace('ID=', ''))
summary_90k_df['Name'] = summary_90k_df['Name'].map(lambda x: x.replace('Name=', ''))
summary_90k_df['coverage'] = summary_90k_df['coverage'].map(lambda x: x.replace('coverage=', ''))
summary_90k_df['identity'] = summary_90k_df['identity'].map(lambda x: x.replace('identity=', ''))

In [4]:
# Generates allele options in format required for hapmap by finding all unique bases in row
# Slaps alleles in dataframe alleles_df
index_col = SUMMARYHEADERS[1]
alleles = []


alleles_df = pd.DataFrame(raw_hap_df[index_col])
rawhap_allele_data_df = raw_hap_df.loc[:, ~raw_hap_df.columns.isin(SUMMARYHEADERS)] # removes all none SNP columns
rawhap_allele_data_list = rawhap_allele_data_df.values.tolist()

for row in rawhap_allele_data_list:
    alleles.append("/".join(set(''.join(row).replace('-', '')))) 
alleles_df['alleles'] = alleles

                             Name alleles
0                   BS00011231_51     A/G
1                   BS00030571_51     A/G
2                   BS00033750_51     G/A
3                   Ex_c6145_2193     A/G
4             Excalibur_c3948_235       G
...                           ...     ...
7349  wsnp_RFL_Contig4207_4836784     G/A
7350                     IACX6482     T/A
7351                     IACX3386     T/A
7352                     IAAV5266     T/A
7353                     IACX6176     T/A

[7354 rows x 2 columns]


In [5]:
# Generate new hapmap file


gen_hap_df = pd.DataFrame()
# For rs#
gen_hap_df[COLHEADERS[0]] = raw_hap_df['Name']
# For alleles
gen_hap_df = gen_hap_df.merge(alleles_df, left_on='rs#', right_on='Name')[COLHEADERS[0:2]]

summary_gen_merge_df = gen_hap_df.merge(summary_90k_df, how="left", left_on='rs#', right_on='Name') # ASSUMPTION: When creating this merge, I assume that the first common "rs#" <-> "Name" found for each row is used, as "Name" is not unique in summary_90k_df
# For chrom
gen_hap_df[COLHEADERS[2]] = summary_gen_merge_df[COLHEADERS[2]]
# Removes unknown chromosomes
gen_hap_df[COLHEADERS[2]].replace('', np.nan, inplace=True)
gen_hap_df.dropna(inplace=True)

# For pos
gen_hap_df[COLHEADERS[3]] = summary_gen_merge_df['pos1'] # Magic value 'pos1' comes from a email suggestion that this column contained relevant position info
gen_hap_df[COLHEADERS[3]] = gen_hap_df[COLHEADERS[3]].astype(int)
# For strand
gen_hap_df[COLHEADERS[4]] = summary_gen_merge_df[COLHEADERS[4]]  
# For other columns not relevant to goal hapmap
gen_hap_df[COLHEADERS[5:len(COLHEADERS)]] = 'NA'

# For snip data
gen_hap_df = gen_hap_df.merge(raw_hap_df, left_on='rs#', right_on='Name')

gen_hap_df.drop(['Name', 'Index'], axis=1, inplace=True)




In [11]:
# Apparently, Hapmaps need to be sorted (ascending) by position 
gen_hap_df = gen_hap_df.sort_values(by=[COLHEADERS[3]])

# Export formatted dataframe to tab seperated csv
gen_hap_df.to_csv(OUTPUT, sep='\t',  index=False)

# The end product of this pipeline was used in Tassel 5 without error (you may need to sort by position in tassel 5)

                              rs# alleles chrom        pos strand assembly#  \
0                   BS00011231_51     A/G     3  685357846      +        NA   
1                   BS00030571_51     A/G     4  602404417      +        NA   
2                   Ex_c6145_2193     A/G     1   12534470      -        NA   
3             Excalibur_c3948_235       G     5   26791983      -        NA   
4           wsnp_JD_c1845_2564931     C/A     5   19439818      -        NA   
...                           ...     ...   ...        ...    ...       ...   
6760  wsnp_RFL_Contig4207_4836784     G/A     4  539758053      +        NA   
6761                     IACX6482     T/A     1  443197960      +        NA   
6762                     IACX3386     T/A     2  637197111      -        NA   
6763                     IAAV5266     T/A     2  576695900      -        NA   
6764                     IACX6176     T/A     2  491951726      -        NA   

     center protLSID assayLSID panelLSID  ... CN116